In [1]:
import pretty_midi
import os

In [2]:
os.chdir("..")

## key transposing

In [3]:
DATA_DIR = "./"

CHROMA_FS = 100

BASS_UPPER_BOUND = pretty_midi.note_name_to_number("Db5")
BASS_LOWER_BOUND = pretty_midi.note_name_to_number("B0")
BASS_REST_THRESH = 0.25

In [4]:
mdid_data = pretty_midi.PrettyMIDI("input_midi/Jazz_ML_MIDI/Jazz Midi/AllofYou.mid")

In [5]:
def find_bass_instrument(midi_data: pretty_midi.PrettyMIDI):
    for instr in midi_data.instruments:
        instr: pretty_midi.Instrument
        if instr.program == 32 and not instr.is_drum:
            return instr
    for instr in midi_data.instruments:
        instr: pretty_midi.Instrument
        if instr.program > 32 and instr.program <= 39 and not instr.is_drum:
            return instr

In [6]:
def transpose_notation(distance):
    if distance == 0:
        return "o0"
    elif distance < 0:
        return "d" + str(abs(distance))
    else:
        return "p" + str(distance)

In [7]:
def find_transpose_range(midi_data: pretty_midi.PrettyMIDI):
    bass_track = find_bass_instrument(midi_data)
    assert bass_track != None, "no bass"
    highest_pitch = max(bass_track.notes, key=lambda note:note.pitch).pitch
    lowest_pitch = min(bass_track.notes, key=lambda note:note.pitch).pitch
    # print(highest_pitch, lowest_pitch)
    transpose_upper_bound = min(6, BASS_UPPER_BOUND-highest_pitch)
    transpose_lower_bound = max(-5, BASS_LOWER_BOUND-lowest_pitch)
    return tuple(range(transpose_lower_bound, transpose_upper_bound+1))

find_transpose_range(mdid_data)

(2, 3, 4, 5, 6)

In [8]:
def transpose_all_instruments(midi_data: pretty_midi.PrettyMIDI, distance):
    for instr in midi_data.instruments:
        instr: pretty_midi.Instrument
        if instr.is_drum:
            continue
        for note in instr.notes:
            note: pretty_midi.Note
            note.pitch += distance
    return midi_data

In [9]:
def transpose_augmentation(midi_name: str, input_dir, output_dir):
    midi_data = pretty_midi.PrettyMIDI(os.path.join(input_dir, midi_name))
    transpose_range = find_transpose_range(midi_data)
    midi_data = transpose_all_instruments(midi_data, transpose_range[0])
    for d in transpose_range:
        output_midi_name = midi_name[:-4] + "_" + transpose_notation(d) + ".mid"
        midi_data.write(os.path.join(output_dir, output_midi_name))
        midi_data = transpose_all_instruments(midi_data, 1)

In [10]:
INPUT_DIRS = ["input_midi/irealpro_midi/"]
OUTPUT_DIR = "input_midi/irealpro_transposed/"

for input_dir in INPUT_DIRS:
    for file_name in sorted(os.listdir(input_dir)):
        try:
            transpose_augmentation(file_name, input_dir, OUTPUT_DIR)
            print(input_dir + file_name)
        except AssertionError as e:
            print(input_dir + file_name + ": " + str(e))
        except Exception as e:
            print(input_dir + file_name + ": " + str(e))

input_midi/irealpro_midi/502 Blues.mid
input_midi/irealpro_midi/52nd Street Theme.mid
input_midi/irealpro_midi/9.20 Special.mid
input_midi/irealpro_midi/A Beautiful Friendship.mid
input_midi/irealpro_midi/A Certain Smile.mid
input_midi/irealpro_midi/A Child Is Born.mid
input_midi/irealpro_midi/A Fine Romance.mid
input_midi/irealpro_midi/A Foggy Day.mid
input_midi/irealpro_midi/A Kiss To Build A Dream On.mid
input_midi/irealpro_midi/A Pretty Girl Is Like A Melody.mid
input_midi/irealpro_midi/A Sleepin' Bee.mid
input_midi/irealpro_midi/A Taste Of Honey.mid
input_midi/irealpro_midi/A Tisket A Tasket.mid
input_midi/irealpro_midi/A Weaver Of Dreams.mid
input_midi/irealpro_midi/Ac-Cent-Tchu-Ate The Positive.mid
input_midi/irealpro_midi/Across The Alley From The Alamo.mid
input_midi/irealpro_midi/Afro Blue.mid
input_midi/irealpro_midi/Afternoon In Paris.mid
input_midi/irealpro_midi/Ain't Misbehavin'.mid
input_midi/irealpro_midi/Alexander's Ragtime Band.mid
input_midi/irealpro_midi/Alfie's The